# Index Tests
Tests on the creation use data with the following composition:
- test1: 20 3-dimensional points
- test2: 30 3-dimensional points
- test3: 100 4-dimensional points

## Import Vantage Point Tree Classes

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
!pip install import-ipynb

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.


In [2]:
%cd "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/notebooks"

import import_ipynb
from index_creation import VP_Tree, Node

print("\nClass VP_Tree and Node has been imported correctly")

/content/gdrive/.shortcut-targets-by-id/1Lqbq3dboVPe48Cxj_XZyhRs54U4hXhEM/[MIRCV]FoodWebSearch/notebooks
importing Jupyter notebook from index_creation.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Class VP_Tree and Node has been imported correctly


## Initialize Notebook

In [3]:
import numpy as np
import os
from scipy.spatial import distance as d

TEST_PATH = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests"

FEATURES_PATH_TEST_1 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-1.npy"
FEATURES_NAMES_TEST_1 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-1.npy"

FEATURES_PATH_TEST_2 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-2.npy"
FEATURES_NAMES_TEST_2 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-2.npy"

FEATURES_PATH_TEST_3 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-3.npy"
FEATURES_NAMES_TEST_3 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-3.npy"

## Partition By Median Tests

In [ ]:
# Partition By Median Tests 1
data = [("image0", np.array([0,2,1])) , ("image1",np.array([2,3,6])), ("image2",np.array([5,3,2])),
          ("image3",np.array([5,6,4])), ("image4",np.array([5,16,1])), ("image5",np.array([2,6,2])), ("image6",np.array([1,3,1]))]

vantage_point_tree = VP_Tree("Index_Test", 5)
node, s_1, s_2 = vantage_point_tree.partition_by_median(data)

print("Node:", node.pivot)
print("Median:", node.median)
print("Set 1:", s_1)
print("Set 2:", s_2)

Node: ('image1', array([2, 3, 6]))
Median: 5.049509756796392
Set 1: [('image3', array([5, 6, 4])), ('image2', array([5, 3, 2])), ('image5', array([2, 6, 2]))]
Set 2: [('image6', array([1, 3, 1])), ('image0', array([0, 2, 1])), ('image4', array([ 5, 16,  1]))]


In [ ]:
# Partition By Median Tests 2
data = [("img_1",np.array([0,0])), ("img_2",np.array([0,1])), ("img_3",np.array([2,2])), ("img_4",np.array([3,3]))]

vantage_point_tree = VP_Tree("Index_Test", 5)

node, s_1, s_2 = vantage_point_tree.partition_by_median(data)

print("Pivot:", node.pivot)
print("Median:", node.median)
print("Set 1:", s_1)
print("Set 2:", s_2)

Pivot: ('img_3', array([2, 2]))
Median: 2.23606797749979
Set 1: [('img_4', array([3, 3])), ('img_2', array([0, 1]))]
Set 2: [('img_2', array([0, 1])), ('img_1', array([0, 0]))]


## Creation Tests with euclidean distance

In [5]:
# Create Tree Test 1
vantage_point_tree = VP_Tree("Index_Test_1",4)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1,FEATURES_PATH_TEST_1)
result = VP_Tree.print_tree(vantage_point_tree.root, level=1)

print("Result:\n" + result + "\n")

index_name = vantage_point_tree.index_name

path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

print("Saved on:", path)

tree_loaded = VP_Tree.load_vptree(path)

print("\nTree Loaded\n")
result = VP_Tree.print_tree(tree_loaded.root, level=1)
print(result)

Number of data: 21
The max height of the tree is: 3
Tree is building
Building of the tree completed in: 5.468463897705078 s
Result:
id: 0 ('img13', array([ 2,  3, 10]))
	id: 00 ('img6', array([1, 3, 1]))
		id: 000 ('img0', array([0, 2, 1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/000_subset_2.npy
		id: 001 ('img18', array([ 5, 23,  1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/001_subset_2.npy
	id: 01 ('img17', array([25,  6, 41]))
		id: 010 ('img9', array([15, 32, 22]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/010_subset_2.npy
		id: 011 ('img14', array([

In [ ]:
# Create Tree Test 2
vantage_point_tree = VP_Tree("Index_Test_2",4)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_2, FEATURES_PATH_TEST_2)
result = VP_Tree.print_tree(vantage_point_tree.root, level=1)
print("Result:\n" + result + "\n")

path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

print("Saved on:", path)

tree_loaded = VP_Tree.load_vptree(path)

print("\nTree Loaded\n")
result = VP_Tree.print_tree(tree_loaded.root, level=1)
print(result)

In [ ]:
# Create Tree Test 3
vantage_point_tree = VP_Tree("Index_Test_3",6)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_3, FEATURES_PATH_TEST_3)
result = VP_Tree.print_tree(vantage_point_tree.root, level=1)
print("Result:\n" + result + "\n")

path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

print("Saved on:", path)

tree_loaded = VP_Tree.load_vptree(path)

print("\nTree Loaded\n")
result = VP_Tree.print_tree(tree_loaded.root, level=1)
print(result)

In [ ]:
# Create Tree Test 4
vantage_point_tree = VP_Tree("Index_Test_4",4, disk_mode=True,leaves_path=TEST_PATH)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1, FEATURES_PATH_TEST_1)
result = VP_Tree.print_tree(vantage_point_tree.root, level=1, disk_mode=True)

print("Result:\n" + result + "\n")

path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

print("Saved on:", path)

tree_loaded = VP_Tree.load_vptree(path)

print("\nTree Loaded\n")
result = VP_Tree.print_tree(tree_loaded.root, level=1)
print(result)

## Creation Tests with Cosine Similarity

In [ ]:
# Create Tree Test 1
vantage_point_tree = VP_Tree("Index_Test_1_sim",4, disk_mode=True, use_similarity=True)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1,FEATURES_PATH_TEST_1)
result = VP_Tree.print_tree(vantage_point_tree.root, level=1)

print("Result:\n" + result + "\n")

index_name = vantage_point_tree.index_name

path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

print("Saved on:", path)

tree_loaded = VP_Tree.load_vptree(path)

print("\nTree Loaded\n")
result = VP_Tree.print_tree(tree_loaded.root, level=1)
print(result)

Number of data: 21
The max height of the tree is: 3
Tree is building
Building of the tree completed in: 3.6347827911376953 s
Result:
id: 0 ('img20', array([10, 31, 10]))
	id: 00 ('img0', array([0, 2, 1]))
		id: 000 ('img7', array([ 5, 23,  4]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1_sim/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1_sim/leaves_3/000_subset_2.npy
		id: 001 ('img4', array([ 5, 16,  1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1_sim/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1_sim/leaves_3/001_subset_2.npy
	id: 01 ('img3', array([5, 6, 4]))
		id: 010 ('img10', array([77, 61, 46]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1_sim/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1_sim/leaves_3/010_subset_2.npy
		i

## Searching Tests

### Tree with Euclidean Distance

In [8]:
# k-NN search Test 1
vantage_point_tree = VP_Tree("Index_Test",4)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1, FEATURES_PATH_TEST_1)
print(VP_Tree.print_tree(vantage_point_tree.root,1))

query = [0,1,1]
k = 3

print("Query:", query, "\n")

nn, d_nn = vantage_point_tree.knn_search(k, query)

print("\nk-NN search on the tree")
print("Points:", nn)
print("Distances:", d_nn)
print("Distance Computed:", vantage_point_tree.distance_computed)
print("File Accessed:", vantage_point_tree.file_accessed)

print("\n\nSequential Scan on the array")
data = np.load(FEATURES_PATH_TEST_1)
distances = [d.euclidean(point, query) for point in data]
zipped = sorted(zip(data, distances), key=lambda x:x[1])
for element in zipped[0:k]:
  print(element)

Number of data: 21
The max height of the tree is: 3
Tree is building
Building of the tree completed in: 0.05205059051513672 s
id: 0 ('img3', array([5, 6, 4]))
	id: 00 ('img5', array([2, 6, 2]))
		id: 000 ('img0', array([0, 2, 1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test/leaves_3/000_subset_2.npy
		id: 001 ('img4', array([ 5, 16,  1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test/leaves_3/001_subset_2.npy
	id: 01 ('img10', array([77, 61, 46]))
		id: 010 ('img20', array([10, 31, 10]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test/leaves_3/010_subset_2.npy
		id: 011 ('img8', array([223,  23,  61]))
			/co

In [ ]:
# k-NN search Test 2
vantage_point_tree = VP_Tree("Index_Test_3",4)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_2, FEATURES_PATH_TEST_2)
print(VP_Tree.print_tree(vantage_point_tree.root,1))

query = [0,1,1]
k = 3

nn, d_nn = vantage_point_tree.knn_search(k, query)

print("Query:", query)

print("k-NN search on the tree")
print("Points:", nn)
print("Distances:", d_nn)
print("Distance Computed:", vantage_point_tree.distance_computed)
print("File Accessed:", vantage_point_tree.file_accessed)

print("\n\nSequential Scan on the array")
data = np.load(FEATURES_PATH_TEST_2)
distances = [d.euclidean(point, query) for point in data]
zipped = sorted(zip(data, distances), key=lambda x:x[1])
for element in zipped[0:k]:
  print(element)

Number of data: 30
The max height of the tree is: 3
Tree is building
Building of the tree completed in: 6.901199102401733 s
id: 0 ('img_26', array([49., 83., 24.]))
	id: 00 ('img_16', array([56., 97., 40.]))
		id: 000 ('img_23', array([40., 97., 35.]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_3/000_subset_2.npy
		id: 001 ('img_1', array([60.21, 24.  , 26.  ]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_3/001_subset_2.npy
	id: 01 ('img_8', array([98., 27., 22.]))
		id: 010 ('img_29', array([43., 34., 61.]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_3/010_subset_2.np

In [ ]:
# k-NN search Test 3
vantage_point_tree = VP_Tree("Index_Test_3",4)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_3, FEATURES_PATH_TEST_3)
print(VP_Tree.print_tree(vantage_point_tree.root,1))

query = [2,17,26,31]
k = 6

nn, d_nn = vantage_point_tree.knn_search(k, query)

print("Query:", query)

print("k-NN search on the tree")
print("Points:", nn)
print("Distances:", d_nn)
print("Distance Computed:", vantage_point_tree.distance_computed)
print("File Accessed:", vantage_point_tree.file_accessed)

print("\n\nSequential Scan on the array")
data = np.load(FEATURES_PATH_TEST_3)
distances = [d.euclidean(point, query) for point in data]
zipped = sorted(zip(data, distances), key=lambda x:x[1])
for element in zipped[0:k]:
  print(element)

Number of data: 100
The max height of the tree is: 5
Tree is building
Building of the tree completed in: 0.12288379669189453 s
id: 0 ('img_28', array([86, 64, 36, 58]))
	id: 00 ('img_74', array([48, 53, 26, 39]))
		id: 000 ('img_33', array([45, 51, 30, 44]))
			id: 0000 ('img_64', array([80, 68, 43, 41]))
				/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_4/0000_subset_1.npy
				/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_4/0000_subset_2.npy
			id: 0001 ('img_31', array([82, 28, 35, 29]))
				/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_4/0001_subset_1.npy
				/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_4/0001_subset_2.npy
		id: 001 ('img_13', array([51, 84, 68, 72]))
			id: 0010 ('img_60', array([44, 36, 38, 92]))
				/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_4/0010_subset_1.npy
				/content/gdrive/My Drive/[M

In [ ]:
#k-NN search Test 4
vantage_point_tree = VP_Tree("Index_Test_3",4, disk_mode=True)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_3, FEATURES_PATH_TEST_3)

query = [0,1,1,2]
k = 6

nn, d_nn = vantage_point_tree.knn_search(k, query)

print("Query:", query)

print("k-NN search on the tree")
print("Points:", nn)
print("Distances:", d_nn)

print("\n\nSequential Scan on the array")
data = np.load(FEATURES_PATH_TEST_3)
distances = [d.euclidean(point, query) for point in data]
zipped = sorted(zip(data, distances), key=lambda x:x[1])
for element in zipped[0:k]:
  print(element)

Number of data: 100
The max height of the tree is: 5
Tree is building
Building of the tree completed in: 0.11896729469299316 s
Query answered in 0.026964664459228516  s
Query: [0, 1, 1, 2]
k-NN search on the tree
Points: [array(['img_95', array([14, 26, 10, 15])], dtype=object), array(['img_75', array([ 2, 17, 25, 31])], dtype=object), array(['img_45', array([11, 29, 18, 31])], dtype=object), array(['img_20', array([20, 28, 34, 12])], dtype=object), array(['img_20', array([20, 28, 34, 12])], dtype=object), array(['img_80', array([10, 31, 30, 25])], dtype=object)]
Distances: [32.72613634390714, 40.95119045888654, 45.110974274559844, 48.14561246884289, 48.14561246884289, 48.68264577855234]


Sequential Scan on the array
(array([14, 26, 10, 15]), 32.72613634390714)
(array([ 2, 17, 25, 31]), 40.95119045888654)
(array([11, 29, 18, 31]), 45.110974274559844)
(array([20, 28, 34, 12]), 48.14561246884289)
(array([10, 31, 30, 25]), 48.68264577855234)
(array([33, 13, 16, 34]), 49.8196748283246)


### Tree with Cosine Similarity

In [ ]:
# k-NN search Test 1 - Similarity
vantage_point_tree = VP_Tree("Index_Test_1_sim",4, use_similarity=True)
vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1, FEATURES_PATH_TEST_1)

query = [0,2,1]
k = 3

print("Query:", query, "\n")

nn, d_nn = vantage_point_tree.knn_search(k, query)

print("\nk-NN search on the tree")
print("Points:", nn)
print("Distances:", d_nn)

print("\n\nSequential Scan on the array")
data = np.load(FEATURES_PATH_TEST_1)
distances = [1-np.dot(point, query)/(np.linalg.norm(point)*np.linalg.norm(query)) for point in data]
zipped = sorted(zip(data, distances), key=lambda x:x[1])
for element in zipped[0:k]:
  print(element)

NameError: ignored